In [ ]:
import os
import cv2
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

In [ ]:
import os
from zipfile import ZipFile


!pip install kaggle


os.environ['KAGGLE_USERNAME'] = "#########"
os.environ['KAGGLE_KEY'] = "#########"


!kaggle datasets download -d jessicali9530/lfw-dataset


with ZipFile('lfw-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('lfw-dataset')


os.remove('lfw-dataset.zip')


 87% 98.0M/112M [00:00<00:00, 133MB/s]
100% 112M/112M [00:00<00:00, 145MB/s] 


In [ ]:

def extract_features(image_path, model):
    image = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = preprocess(image)
    image = image.unsqueeze(0)
    with torch.no_grad():
        features = model(image)
    features = features.squeeze(0)
    return features


def compute_hog(img):
    resized_img = resize(img, (128*4, 64*4))
    fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2), visualize=True, multichannel=True)
    return fd


def get_pixel(img, center, x, y):
    new_value = 0
    try:
        if img[x][y] >= center:
            new_value = 1
    except:
        pass
    return new_value

def lbp_calculated_pixel(img, x, y):
    center = img[x][y]
    val_ar = []
    val_ar.append(get_pixel(img, center, x-1, y+1))
    val_ar.append(get_pixel(img, center, x, y+1))
    val_ar.append(get_pixel(img, center, x+1, y+1))
    val_ar.append(get_pixel(img, center, x+1, y))
    val_ar.append(get_pixel(img, center, x+1, y-1))
    val_ar.append(get_pixel(img, center, x, y-1))
    val_ar.append(get_pixel(img, center, x-1, y-1))
    val_ar.append(get_pixel(img, center, x-1, y))

    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
    val = 0
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
    return val

def calcLBP(img):
    height, width, channel = img.shape
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_lbp = np.zeros((height, width,3), np.uint8)
    for i in range(0, height):
        for j in range(0, width):
             img_lbp[i, j] = lbp_calculated_pixel(img_gray, i, j)
    hist_lbp = cv2.calcHist([img_lbp], [0], None, [256], [0, 256])
    return hist_lbp.flatten()


In [ ]:
resnet = models.resnet50(pretrained=True)
resnet = nn.Sequential(*list(resnet.children())[:-1])
resnet.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 80.0MB/s]


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [ ]:
lfw_folder = '/content/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled'
X, y = [], []
for folder_name in os.listdir(lfw_folder):
    folder_path = os.path.join(lfw_folder, folder_name)
    if os.path.isdir(folder_path):
        num_images = len(os.listdir(folder_path))
        if num_images > 70:
            for image_name in os.listdir(folder_path):
                image_path = os.path.join(folder_path, image_name)
                image = imread(image_path)
                hog_feature = compute_hog(image)
                lbp_feature = calcLBP(image)
                cnn_feature = extract_features(image_path, resnet).numpy()
                hog_feature = hog_feature.reshape(-1)
                lbp_feature = lbp_feature.reshape(-1)
                cnn_feature = cnn_feature.flatten()
                combined_feature = np.concatenate((hog_feature, lbp_feature, cnn_feature))

                X.append(combined_feature)
                y.append(folder_name)


<ipython-input-3-6e69d5230bcf>:20: FutureWarning: `multichannel` is a deprecated argument name for `hog`. It will be removed in version 1.0. Please use `channel_axis` instead.
  fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)



In [ ]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [ ]:
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_train_encoded)
y_pred_knn = knn_clf.predict(X_test)
accuracy_knn = accuracy_score(y_test_encoded, y_pred_knn)
print("KNN Classifier")
print("Accuracy:", accuracy_knn)
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_knn, target_names=label_encoder.classes_))


KNN Classifier
Accuracy: 0.3643410852713178
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       0.17      0.10      0.12        21
     Colin_Powell       0.23      0.37      0.28        41
  Donald_Rumsfeld       0.12      0.10      0.11        20
    George_W_Bush       0.51      0.60      0.55       114
Gerhard_Schroeder       0.18      0.13      0.15        23
      Hugo_Chavez       0.00      0.00      0.00        10
       Tony_Blair       0.40      0.14      0.21        29

         accuracy                           0.36       258
        macro avg       0.23      0.20      0.20       258
     weighted avg       0.34      0.36      0.34       258



In [ ]:
svm_clf = SVC()
svm_clf.fit(X_train, y_train_encoded)
y_pred_svm = svm_clf.predict(X_test)
accuracy_svm = accuracy_score(y_test_encoded, y_pred_svm)
print("SVM Classifier")
print("Accuracy:", accuracy_svm)
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_svm, target_names=label_encoder.classes_))


SVM Classifier
Accuracy: 0.46124031007751937
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       0.00      0.00      0.00        21
     Colin_Powell       0.38      0.37      0.37        41
  Donald_Rumsfeld       0.00      0.00      0.00        20
    George_W_Bush       0.47      0.91      0.62       114
Gerhard_Schroeder       0.00      0.00      0.00        23
      Hugo_Chavez       0.00      0.00      0.00        10
       Tony_Blair       0.00      0.00      0.00        29

         accuracy                           0.46       258
        macro avg       0.12      0.18      0.14       258
     weighted avg       0.27      0.46      0.34       258



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train_encoded)
y_pred_dt = dt_clf.predict(X_test)
accuracy_dt = accuracy_score(y_test_encoded, y_pred_dt)
print("Decision Tree Classifier")
print("Accuracy:", accuracy_dt)
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_dt, target_names=label_encoder.classes_))


Decision Tree Classifier
Accuracy: 0.5736434108527132
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       0.71      0.48      0.57        21
     Colin_Powell       0.53      0.51      0.52        41
  Donald_Rumsfeld       0.36      0.45      0.40        20
    George_W_Bush       0.74      0.70      0.72       114
Gerhard_Schroeder       0.50      0.48      0.49        23
      Hugo_Chavez       0.07      0.10      0.08        10
       Tony_Blair       0.47      0.55      0.51        29

         accuracy                           0.57       258
        macro avg       0.48      0.47      0.47       258
     weighted avg       0.60      0.57      0.58       258



In [ ]:
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train_encoded)
y_pred_rf = rf_clf.predict(X_test)
accuracy_rf = accuracy_score(y_test_encoded, y_pred_rf)
print("Random Forest Classifier")
print("Accuracy:", accuracy_rf)
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_rf, target_names=label_encoder.classes_))


Random Forest Classifier
Accuracy: 0.6976744186046512
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       0.00      0.00      0.00        21
     Colin_Powell       0.90      0.93      0.92        41
  Donald_Rumsfeld       1.00      0.45      0.62        20
    George_W_Bush       0.61      1.00      0.75       114
Gerhard_Schroeder       1.00      0.26      0.41        23
      Hugo_Chavez       0.00      0.00      0.00        10
       Tony_Blair       1.00      0.45      0.62        29

         accuracy                           0.70       258
        macro avg       0.64      0.44      0.47       258
     weighted avg       0.69      0.70      0.63       258



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train_encoded)
y_pred_lr = lr_clf.predict(X_test)
accuracy_lr = accuracy_score(y_test_encoded, y_pred_lr)
print("Logistic Regression Classifier")
print("Accuracy:", accuracy_lr)
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_lr, target_names=label_encoder.classes_))


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Classifier
Accuracy: 0.42248062015503873
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       0.50      0.19      0.28        21
     Colin_Powell       0.23      0.32      0.27        41
  Donald_Rumsfeld       0.10      0.05      0.07        20
    George_W_Bush       0.54      0.77      0.63       114
Gerhard_Schroeder       0.00      0.00      0.00        23
      Hugo_Chavez       0.00      0.00      0.00        10
       Tony_Blair       0.20      0.10      0.14        29

         accuracy                           0.42       258
        macro avg       0.22      0.20      0.20       258
     weighted avg       0.34      0.42      0.36       258



In [ ]:
nb_clf = GaussianNB()
nb_clf.fit(X_train, y_train_encoded)
y_pred_nb = nb_clf.predict(X_test)
accuracy_nb = accuracy_score(y_test_encoded, y_pred_nb)
print("Naive Bayes Classifier")
print("Accuracy:", accuracy_nb)
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_nb, target_names=label_encoder.classes_))


Naive Bayes Classifier
Accuracy: 0.46124031007751937
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       0.72      0.62      0.67        21
     Colin_Powell       0.45      0.68      0.54        41
  Donald_Rumsfeld       0.21      1.00      0.34        20
    George_W_Bush       1.00      0.27      0.43       114
Gerhard_Schroeder       0.80      0.35      0.48        23
      Hugo_Chavez       1.00      0.30      0.46        10
       Tony_Blair       0.42      0.55      0.48        29

         accuracy                           0.46       258
        macro avg       0.66      0.54      0.49       258
     weighted avg       0.75      0.46      0.47       258



In [ ]:
mlp_clf = MLPClassifier()
mlp_clf.fit(X_train, y_train_encoded)
y_pred_mlp = mlp_clf.predict(X_test)
accuracy_mlp = accuracy_score(y_test_encoded, y_pred_mlp)
print("MLP Classifier")
print("Accuracy:", accuracy_mlp)
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_mlp, target_names=label_encoder.classes_))

MLP Classifier
Accuracy: 0.8255813953488372
Classification Report:
                   precision    recall  f1-score   support

     Ariel_Sharon       1.00      0.52      0.69        21
     Colin_Powell       0.90      0.90      0.90        41
  Donald_Rumsfeld       0.61      0.70      0.65        20
    George_W_Bush       0.86      0.97      0.91       114
Gerhard_Schroeder       0.93      0.61      0.74        23
      Hugo_Chavez       0.29      0.50      0.37        10
       Tony_Blair       0.95      0.72      0.82        29

         accuracy                           0.83       258
        macro avg       0.79      0.70      0.73       258
     weighted avg       0.85      0.83      0.83       258

